In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

print(tf.__version__)

1.12.0


In [2]:
#this is just to get the data for demo purposes
from tensorflow import keras
dataset_path = keras.utils.get_file("C:\\development\\local_dev\\mpg_serving\\auto-mpg.data", "https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data")
dataset_path

32768/30286 [================================] - 0s 3us/step


'C:\\development\\local_dev\\mpg_serving\\auto-mpg.data'

In [3]:
column_names = ['MPG','Cylinders','Displacement','Horsepower','Weight',
                'Acceleration', 'Model_Year', 'Origin'] 
raw_dataset = pd.read_csv(dataset_path, names=column_names,
                      na_values = "?", comment='\t',
                      sep=" ", skipinitialspace=True)

dataset = raw_dataset.copy()
dataset.tail()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model_Year,Origin
393,27.0,4,140.0,86.0,2790.0,15.6,82,1
394,44.0,4,97.0,52.0,2130.0,24.6,82,2
395,32.0,4,135.0,84.0,2295.0,11.6,82,1
396,28.0,4,120.0,79.0,2625.0,18.6,82,1
397,31.0,4,119.0,82.0,2720.0,19.4,82,1


In [4]:
dataset.isna().sum()

MPG             0
Cylinders       0
Displacement    0
Horsepower      6
Weight          0
Acceleration    0
Model_Year      0
Origin          0
dtype: int64

In [5]:
dataset = dataset.dropna()

In [6]:
dataset['USA'] = (dataset['Origin'] == 1)*1.0
dataset['Europe'] = (dataset['Origin'] == 2)*1.0
dataset['Japan'] = (dataset['Origin'] == 3)*1.0
dataset.tail()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model_Year,Origin,USA,Europe,Japan
393,27.0,4,140.0,86.0,2790.0,15.6,82,1,1.0,0.0,0.0
394,44.0,4,97.0,52.0,2130.0,24.6,82,2,0.0,1.0,0.0
395,32.0,4,135.0,84.0,2295.0,11.6,82,1,1.0,0.0,0.0
396,28.0,4,120.0,79.0,2625.0,18.6,82,1,1.0,0.0,0.0
397,31.0,4,119.0,82.0,2720.0,19.4,82,1,1.0,0.0,0.0


In [7]:
train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)

In [8]:
train_stats = train_dataset.describe()
train_stats.pop("MPG")
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
Cylinders,314.0,5.477707,1.699788,3.0,4.00,4.0,8.00,8.0
Displacement,314.0,195.318471,104.331589,68.0,105.50,151.0,265.75,455.0
Horsepower,314.0,104.869427,38.096214,46.0,76.25,94.5,128.00,225.0
Weight,314.0,2990.251592,843.898596,1649.0,2256.50,2822.5,3608.00,5140.0
Acceleration,314.0,15.559236,2.789230,8.0,13.80,15.5,17.20,24.8
Model_Year,314.0,75.898089,3.675642,70.0,73.00,76.0,79.00,82.0
Origin,314.0,1.573248,0.800988,1.0,1.00,1.0,2.00,3.0
USA,314.0,0.624204,0.485101,0.0,0.00,1.0,1.00,1.0
Europe,314.0,0.178344,0.383413,0.0,0.00,0.0,0.00,1.0
Japan,314.0,0.197452,0.398712,0.0,0.00,0.0,0.00,1.0


In [9]:
train_labels = train_dataset.pop('MPG')
test_labels = test_dataset.pop('MPG')

In [10]:
def norm(x):
    return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

In [11]:
feat_cols = [
    tf.feature_column.numeric_column(key="Cylinders"),
    tf.feature_column.numeric_column(key="Displacement"),
    tf.feature_column.numeric_column(key="Horsepower"),
    tf.feature_column.numeric_column(key="Acceleration"),
    tf.feature_column.numeric_column(key="Model_Year"),
    tf.feature_column.numeric_column(key="USA"),
    tf.feature_column.numeric_column(key="Europe"),
    tf.feature_column.numeric_column(key="Japan"),
]

In [12]:
input_func = tf.estimator.inputs.pandas_input_fn(x=normed_train_data,y=train_labels,
                                                 batch_size=10,num_epochs=1000,shuffle=True)

In [13]:
model = tf.estimator.DNNRegressor(hidden_units=[64,64,1],feature_columns=feat_cols,model_dir='demo')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'demo', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001F8F944A390>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [14]:
model.train(input_fn=input_func,steps=5000)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into demo\model.ckpt.
INFO:tensorflow:loss = 4965.0996, step = 1
INFO:tensorflow:global_step/sec: 403.231
INFO:tensorflow:loss = 46.883, step = 101 (0.251 sec)
INFO:tensorflow:global_step/sec: 448.434
INFO:tensorflow:loss = 64.186745, step = 201 (0.221 sec)
INFO:tensorflow:global_step/sec: 413.185
INFO:tensorflow:loss = 30.499115, step = 301 (0.245 sec)
INFO:tensorflow:global_step/sec: 404.899
INFO:tensorflow:loss = 53.8582, step = 401 (0.245 sec)
INFO:tensorflow:globa

In [15]:
input_func_eval = tf.estimator.inputs.pandas_input_fn(x=normed_test_data,y=test_labels,batch_size=10,shuffle=False)

In [16]:
eval_results = model.evaluate(input_fn=input_func_eval)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-01-22-18:18:25
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from demo\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-01-22-18:18:25
INFO:tensorflow:Saving dict for global step 5000: average_loss = 7.349957, global_step = 5000, label/mean = 23.991026, loss = 71.66208, prediction/mean = 24.429272
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: demo\model.ckpt-5000


In [17]:
def normalize_input_data(a, value):
    a_mean = train_stats['mean'][(a)]
    #print(a_mean)
    a_std = train_stats['std'][(a)]
    #print(a_std)
    #print(value)
    return (value - a_mean) / a_std

In [18]:
def serving_input_receiver_fn():
    
    #The below code is right from the TF guide except for the first line
    feature_spec = tf.feature_column.make_parse_example_spec(feat_cols)
    default_batch_size = 1
    serialized_tf_example = tf.placeholder(dtype=tf.string, shape=[default_batch_size], name='input_example_tensor')
    receiver_tensors = {'examples': serialized_tf_example}
    features = tf.parse_example(serialized_tf_example, feature_spec)
    
    fn_Cylinders = lambda Cylinders: normalize_input_data('Cylinders', Cylinders)
    fn_Displacement = lambda Displacement: normalize_input_data('Displacement', Displacement)
    fn_Horsepower = lambda Horsepower: normalize_input_data('Horsepower', Horsepower)
    fn_Acceleration = lambda Acceleration: normalize_input_data('Acceleration', Acceleration)
    fn_Model_Year = lambda Model_Year: normalize_input_data('Model_Year', Model_Year)
    fn_USA = lambda USA: normalize_input_data('USA', USA)
    fn_Europe = lambda Europe: normalize_input_data('Europe', Europe)
    fn_Japan = lambda Japan: normalize_input_data('Japan', Japan)
    
    features['Cylinders'] = tf.map_fn(fn_Cylinders, features['Cylinders'], dtype=tf.float32)
    features['Displacement'] = tf.map_fn(fn_Displacement, features['Displacement'], dtype=tf.float32)
    features['Horsepower'] = tf.map_fn(fn_Horsepower, features['Horsepower'], dtype=tf.float32)
    features['Acceleration'] = tf.map_fn(fn_Acceleration, features['Acceleration'], dtype=tf.float32)
    features['Model_Year'] = tf.map_fn(fn_Model_Year, features['Model_Year'], dtype=tf.float32)
    features['USA'] = tf.map_fn(fn_USA, features['USA'], dtype=tf.float32)
    features['Europe'] = tf.map_fn(fn_Europe, features['Europe'], dtype=tf.float32)
    features['Japan'] = tf.map_fn(fn_Japan, features['Japan'], dtype=tf.float32)
    
    return tf.estimator.export.ServingInputReceiver(features, receiver_tensors)

In [19]:
model.export_savedmodel('exports_dev', serving_input_receiver_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: ['serving_default', 'regression']
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from demo\model.ckpt-5000
Instructions for updating:
Pass your op to the equivalent parameter main_op instead.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: exports_dev\temp-b'1548181106'\saved_model.pb


b'exports_dev\\1548181106'